In [76]:
import simpy
import random
import pandas as pd

# Constants
RANDOM_SEED = 42
NEW_PATIENTS = 25  # Total number of patients for the simulation
INTERVAL_PATIENTS = 10.0  # Generate new patients roughly every 10 minutes
SIM_TIME = 360  # Simulation time in minutes
TREATMENT_TIME = 15  # Average time a patient spends with the doctor
LAB_TEST_TIME = 10  # Average time for conducting lab tests
RESULTS_WAIT_TIME = 20  # Time to wait for lab results
FOLLOW_UP_TIME = 5  # Time for follow-up consultation

NUM_DOCTORS = 100  # Number of available doctors
NUM_NURSES = 100  # Number of available nurses
NUM_LAB_TECHS = 100  # Number of available lab technicians
NUM_EXAM_ROOMS = 100  # Number of available exam rooms

# Initialize a list to store patient records
patient_records = []

class EmergencyDepartment:
    def __init__(self, env, num_doctors, num_nurses, num_lab_techs, num_exam_rooms):
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)
        self.nurse = simpy.Resource(env, num_nurses)
        self.lab = simpy.Resource(env, num_lab_techs)
        self.exam_room = simpy.Resource(env, num_exam_rooms)

    def triage(self, patient):
        yield self.env.timeout(random.randint(1, 3))  # Triage takes 1-3 minutes

    def treatment(self, patient):
        yield self.env.timeout(random.randint(5, TREATMENT_TIME))  # Treatment time

    def lab_test(self, patient):
        yield self.env.timeout(random.randint(5, LAB_TEST_TIME))  # Lab test time

    def follow_up(self, patient):
        yield self.env.timeout(FOLLOW_UP_TIME)  # Follow-up consultation time

def patient(env, name, ed):
    arrival_time = env.now
    print(f'{name} arrives at the emergency department at {env.now:.1f}')
    with ed.nurse.request() as request:
        yield request
        triage_start_time = env.now
        yield env.process(ed.triage(name))
        triage_end_time = env.now
        # Nurse decision to admit directly with 5% chance
        if random.random() < 0.05:
            print(f'{name} is directly admitted to the hospital after triage at {env.now:.1f}')
            patient_records.append({
                'Patient': name,
                'Arrival Time': arrival_time,
                'Triage Start': triage_start_time,
                'Triage End': triage_end_time,
                'Outcome': 'Admitted by Nurse'
            })
            return  # Ends the process for this patient as they are admitted
        print(f'{name} leaves triage at {env.now:.1f}')

    with ed.exam_room.request() as request:
        yield request
        with ed.doctor.request() as doc_request:
            yield doc_request
            treatment_start_time = env.now
            yield env.process(ed.treatment(name))
            treatment_end_time = env.now
            print(f'{name} needs lab tests at {env.now:.1f}')

    with ed.lab.request() as lab_request:
        yield lab_request
        lab_start_time = env.now
        yield env.process(ed.lab_test(name))
        lab_end_time = env.now
        print(f'{name} lab test completed at {env.now:.1f}')
    
    # Wait for results
    results_ready_time = env.now + RESULTS_WAIT_TIME
    yield env.timeout(RESULTS_WAIT_TIME)
    print(f'{name} lab results are ready at {env.now:.1f}')

    with ed.doctor.request() as doc_request:
        yield doc_request
        follow_up_start_time = env.now
        yield env.process(ed.follow_up(name))
        follow_up_end_time = env.now
        # Decision making for discharge or admission
        if random.random() < 0.7:  # 70% chance to discharge, 30% to admit
            outcome = 'Discharged'
        else:
            outcome = 'Admitted'
        print(f'{name} {outcome} at {env.now:.1f}')
        patient_records.append({
            'Patient': name,
            'Arrival Time': arrival_time,
            'Triage Start': triage_start_time,
            'Triage End': triage_end_time,
            'Treatment Start': treatment_start_time,
            'Treatment End': treatment_end_time,
            'Lab Test Start': lab_start_time,
            'Lab Test End': lab_end_time,
            'Results Ready': results_ready_time,
            'Follow Up Start': follow_up_start_time,
            'Follow Up End': follow_up_end_time,
            'Outcome': outcome
        })

def setup(env, num_doctors, num_nurses, num_lab_techs, num_exam_rooms):
    emergency_department = EmergencyDepartment(env, num_doctors, num_nurses, num_lab_techs, num_exam_rooms)

    for i in range(NEW_PATIENTS):
        env.process(patient(env, f'Patient {i}', emergency_department))

    while True:
        yield env.timeout(random.expovariate(1.0 / INTERVAL_PATIENTS))
        i += 1
        env.process(patient(env, f'Patient {i}', emergency_department))

# Setup and start the simulation
print('Emergency Department Simulation')
random.seed(RANDOM_SEED)
env = simpy.Environment()
env.process(setup(env, num_doctors=NUM_DOCTORS, num_nurses=NUM_NURSES, num_lab_techs=NUM_LAB_TECHS, num_exam_rooms=NUM_EXAM_ROOMS))
env.run(until=SIM_TIME)

# Convert the list of patient records into a DataFrame
df_patients = pd.DataFrame(patient_records)


Emergency Department Simulation
Patient 0 arrives at the emergency department at 0.0
Patient 1 arrives at the emergency department at 0.0
Patient 2 arrives at the emergency department at 0.0
Patient 3 arrives at the emergency department at 0.0
Patient 4 arrives at the emergency department at 0.0
Patient 5 arrives at the emergency department at 0.0
Patient 6 arrives at the emergency department at 0.0
Patient 7 arrives at the emergency department at 0.0
Patient 8 arrives at the emergency department at 0.0
Patient 9 arrives at the emergency department at 0.0
Patient 10 arrives at the emergency department at 0.0
Patient 11 arrives at the emergency department at 0.0
Patient 12 arrives at the emergency department at 0.0
Patient 13 arrives at the emergency department at 0.0
Patient 14 arrives at the emergency department at 0.0
Patient 15 arrives at the emergency department at 0.0
Patient 16 arrives at the emergency department at 0.0
Patient 17 arrives at the emergency department at 0.0
Patien

In [77]:
df_patients

,Patient,Arrival Time,Triage Start,Triage End,Outcome,Treatment Start,Treatment End,Lab Test Start,Lab Test End,Results Ready,Follow Up Start,Follow Up End
0,Patient 11,0.000000,0.000000,1.000000,Admitted by Nurse,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Patient 0,0.000000,0.000000,1.000000,Admitted,1.000000,7.000000,7.000000,13.000000,33.000000,33.000000,38.000000
2,Patient 22,0.000000,0.000000,3.000000,Discharged,3.000000,9.000000,9.000000,15.000000,35.000000,35.000000,40.000000
3,Patient 14,0.000000,0.000000,1.000000,Discharged,1.000000,6.000000,6.000000,16.000000,36.000000,36.000000,41.000000
4,Patient 17,0.000000,0.000000,1.000000,Admitted,1.000000,7.000000,7.000000,16.000000,36.000000,36.000000,41.000000
5,Patient 7,0.000000,0.000000,1.000000,Discharged,1.000000,10.000000,10.000000,16.000000,36.000000,36.000000,41.000000
6,Patient 21,0.000000,0.000000,1.000000,Discharged,1.000000,7.000000,7.000000,17.000000,37.000000,37.000000,42.000000
7,Patient 9,0.000000,0.000000,3.000000,Discharged,3.000000,11.000000,11.000000,17.000000,37.000000,37.000000,42.000000
8,Patient 12,0.000000,0.000000,3.000000,Discharged,3.000000,12.000000,12.000000,17.000000,37.000000,37.000000,42.000000
9,Patient 1,0.000000,0.000000,3.000000,Discharged,3.000000,10.000000,10.000000,18.000000,38.000000,38.000000,43.000000


In [78]:
print("Number of Patient Records:", len(df_patients))

Number of Patient Records: 59


In [79]:
df_discharged = df_patients[df_patients['Outcome'] == 'Discharged']
df_discharged = df_discharged.reset_index()
df_discharged['Total Time'] = df_discharged['Follow Up End'] - df_discharged['Arrival Time']
print("Number of Patients Discharged", len(df_discharged))
print("Average Time in Emergency Department for Discharged Patients:", df_discharged['Total Time'].mean())

Number of Patients Discharged 36
Average Time in Emergency Department for Discharged Patients: 44.52777777777778


In [80]:
df_admitted = df_patients[df_patients['Outcome'] == 'Admitted']
df_admitted = df_admitted.reset_index()
df_admitted['Total Time'] = df_admitted['Follow Up End'] - df_admitted['Arrival Time']
print("Number of Patients Admitted", len(df_admitted))
print("Average Time in Emergency Department for Admitted Patients:", df_admitted['Total Time'].mean())

Number of Patients Admitted 20
Average Time in Emergency Department for Admitted Patients: 45.65
